# Getting information from the batteries database

There are currently no pymatgen MPRester methods to retrieve battery information, but there is an API endpoint to retrieve such information by chemical formula or battery ID. Here is a way to add such a method in your code:

In [1]:
from pymatgen import MPRester
import pandas as pd

mpr = MPRester()

def get_battery_data(self, formula_or_batt_id):
    """Returns batteries from a batt id or formula.

    Examples:
        get_battery("mp-300585433")
        get_battery("LiFePO4")
    """
    return mpr._make_request('/battery/%s' % formula_or_batt_id)
MPRester.get_battery_data = get_battery_data

/Users/lik/miniconda3/envs/pmg/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/lik/miniconda3/envs/pmg/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## You can search by name or batt-ID

The function returns a list (of one if you search by ID) of battery database entries.

In [2]:
# If you search by formula you might get multiple entries
results_formula = mpr.get_battery_data("LiV5O12")
print([itr['battid'] for itr in results_formula])

['mp-300776654', 'mp-300851075', 'mp-300777675']


In [3]:
# Or you can search by batt-ID
results_battid = mpr.get_battery_data("mp-300776654") # Li0-7V5O12
print([itr['battid'] for itr in results_battid])

['mp-300776654']


In [4]:
# Each charge discharge pair in the database is strored in 'adj_pairs'
["{} --> {}".format(itr['id_discharge'], itr['id_charge']) for itr in results_battid[0]['adj_pairs']]
# Note the the items charge/discharge occurs sequentially

['mp-762254 --> mp-774289',
 'mp-762281 --> mp-762254',
 'mp-764771 --> mp-762281',
 'mp-776654 --> mp-764771']

## Basic usage case

- You have a list of batteries matterials you looked at on the website that you really liked
- You want to tabulate some basic properties of those materials and store them in a spreadsheet

In [5]:
names=["mp-300019408","mp-300773362","mp-300850769","mp-300038280","mp-300772030",
       "mp-300003924","mp-300764772","mp-300019340","mp-300018793","mp-300766115",
       "mp-300762454","mp-300018767","mp-300775236","mp-300019279","mp-300765516",
       "mp-300771995","mp-300018782","mp-300774096","mp-300851027","mp-300763684",
       "mp-300769436","mp-300775097","mp-300024850","mp-300018925","mp-300553966",
       "mp-300545467","mp-300018162","mp-300771617","mp-300774096","mp-300851027",
       "mp-300019511","mp-300540112","mp-300031869","mp-300031800","mp-300031583",
       "mp-300769828","mp-300018925","mp-300763301","mp-300763301","mp-300025660",
       "mp-300019308","mp-300771725","mp-300775737","mp-300566811","mp-300699330"]

In [6]:
# you can filter for the list of properties that you are interested in a given 'adj_pair'
props=['formula_charge', 'formula_discharge', 'stability_charge', 
       'stability_discharge','id_charge','id_discharge']
{k:v for k,v in results_battid[0]['adj_pairs'][0].items() if k in props}

{'id_discharge': 'mp-762254',
 'formula_discharge': 'Li3V5O12',
 'stability_discharge': 0.07312749874999991,
 'stability_charge': 0.09947182705882263,
 'formula_charge': 'V5O12',
 'id_charge': 'mp-774289'}

In [7]:
def get_data(batt_ID):
    #result = mpr.get_battery_data(mat_name)
    result = mpr.get_battery_data(batt_ID)
    d = {k:v for k,v in result[0]['adj_pairs'][0].items() if k in props}
    d['battid'] = batt_ID
    return d

In [8]:
df = pd.DataFrame(columns=props)
for iname in names:
    d = get_data(iname)
    df = df.append(d,ignore_index=True)

In [9]:
df

,formula_charge,formula_discharge,stability_charge,stability_discharge,id_charge,id_discharge,battid
0,V2O5,LiV2O5,0.010801,0.024130,mp-25643,mp-19408,mp-300019408
1,V2O5,LiV4O10,0.030147,0.030225,mp-25646,mp-776037,mp-300773362
2,V2O5,Li9V14O35,0.027676,0.051649,mp-849565,mp-850769,mp-300850769
3,TiO2,LiTiO2,0.006060,0.000000,mp-390,mp-38280,mp-300038280
4,TiO2,Li3Ti4O8,0.181589,0.046079,mvc-11115,mp-761891,mp-300772030
5,NbO2,LiNb2O4,0.283840,0.059403,mp-25214,mp-758673,mp-300003924
6,LiV5O10,Li2V5O10,0.079549,0.070749,mp-762390,mp-762287,mp-300764772
7,VO2,Li2V3O6,0.144599,0.068063,mp-714944,mp-762271,mp-300019340
8,CrO2,Li3Cr5O10,0.191242,0.062740,mp-714929,mp-762282,mp-300018793
9,CrO2,Li2Cr3O6,0.182603,0.068596,mvc-9788,mp-762466,mp-300766115
